In [24]:
import pandas as pd
import numpy as np
import psycopg2
from sql import engine
import h3

pa_data = pd.read_csv('./data/Charging_PA.csv',sep=',')

/usr/local/Caskroom/miniconda/base/envs/sql-practice/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (29,30,32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [25]:
charging_df = pd.DataFrame(pa_data)

In [26]:
charging_df.drop(['Start Time Zone','End Time Zone', 'Transaction Date (Pacific Time)',
                'GHG Savings (kg)', 'Gasoline Savings (gallons)','Address 1','City', 
                'State/Province', 'Postal Code', 'Country', 'County', 'System S/N', 
                'Model Number','Org Name','EVSE ID','Energy (kWh)','Plug Type','Currency'], inplace= True, axis= 1)

In [27]:
charging_df.columns = charging_df.columns.str.replace(' ', '_')
charging_df.columns = charging_df.columns.str.lower()
charging_df.columns = charging_df.columns.str.rstrip()

In [28]:
charging_df = charging_df.rename(columns={'total_duration_(hh:mm:ss)': 'total_duration', "charging_time_(hh:mm:ss)": "charging_time"})

In [29]:
charging_2020 = charging_df.query('start_date >= "2020-01-01" & start_date <= "2020-12-31"')
charging_2019 = charging_df.query('start_date >= "2019-01-01" & start_date <= "2019-12-31"')
charging_2018 = charging_df.query('start_date >= "2018-01-01" & start_date <= "2018-12-31"')
charging_2017 = charging_df.query('start_date >= "2017-01-01" & start_date <= "2017-12-31"')
charging_2016 = charging_df.query('start_date >= "2016-01-01" & start_date <= "2016-12-31"')
charging_2015 = charging_df.query('start_date >= "2015-01-01" & start_date <= "2015-12-31"')
charging_2014 = charging_df.query('start_date >= "2014-01-01" & start_date <= "2014-12-31"')
charging_2013 = charging_df.query('start_date >= "2013-01-01" & start_date <= "2013-12-31"')
charging_2012 = charging_df.query('start_date >= "2012-01-01" & start_date <= "2012-12-31"')
charging_2011 = charging_df.query('start_date >= "2011-01-01" & start_date <= "2011-12-31"')

In [30]:
# Make datetime columns only for 2019
date_cols = ["start_date", "end_date"]
duration_cols = ["total_duration", "charging_time"]

for col in date_cols:
     charging_2019[date_cols] = charging_2019[date_cols].apply(pd.to_datetime, errors = "coerce")


for col in duration_cols:
     charging_2019[duration_cols] = charging_2019[duration_cols].apply(pd.to_timedelta, errors="coerce")

In [35]:
charging_2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   station_name        0 non-null      object 
 1   mac_address         0 non-null      object 
 2   start_date          0 non-null      object 
 3   end_date            0 non-null      object 
 4   total_duration      0 non-null      object 
 5   charging_time       0 non-null      object 
 6   port_type           0 non-null      object 
 7   port_number         0 non-null      int64  
 8   latitude            0 non-null      float64
 9   longitude           0 non-null      float64
 10  ended_by            0 non-null      object 
 11  plug_in_event_id    0 non-null      int64  
 12  driver_postal_code  0 non-null      float64
 13  user_id             0 non-null      object 
 14  fee_bol             0 non-null      bool   
 15  parking_time        0 non-null      object 
dtypes: bool(1), float64(

In [31]:
# replace NaT in end_date with start_date + total_duration
charging_2019['end_date'] = charging_2019['end_date'].fillna((charging_2019['start_date'] + charging_2019['total_duration']))

In [32]:
# create column fee_bol (True if user paid, False if charging was free), drop original fee column
charging_2019['fee_bol'] = np.where(charging_2019['fee']== 0.0, False, True)
charging_2019.drop(['fee'],inplace= True, axis= 1)

/usr/local/Caskroom/miniconda/base/envs/sql-practice/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [33]:
# create colum parking_time to display parking time (plugged in but not charging)
charging_2019['parking_time'] = charging_2019['total_duration'] - charging_2019['charging_time']

Which year do we want to analyze?

unique identifier for each station plug = combination of plug ID and station name

In [34]:
# create column that displays weekday based on start_date
charging_2019['weekday'] = charging_2019['start_date'].dt.dayofweek

AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
charging_2019['weekday'].unique()
mymapp = {'0': 'Monday', '1': 'Tuesday', '2': 'Wednesday', '3': 'Thursday', '4': 'Friday', '5': 'Saturday', '6': 'Sunday'}
charging_2019.applymap(lambda s: mymapp.get(s) if s in mymapp else s)

In [ ]:
def make_weekday_names(row):
    if row['weekday'] == 0:
        return 'Monday'
    if row['weekday'] == 1:
        return 'Tuesday'
    if row['weekday'] == 2:
        return 'Wednesday'
    if row['weekday'] == 3:
        return 'Thursday'
    if row['weekday'] == 4:
        return 'Friday'
    if row['weekday'] == 5:
        return 'Saturday'
    if row['weekday'] == 6:
        return 'Sunday'

In [ ]:
charging_2019['weekday'] = charging_2019.apply(lambda row : make_weekday_names(row), axis=1)

In [ ]:
def make_mac_same(row):
    return row['mac_address'].replace(':','')

charging_2019['mac_address'] = charging_2019.apply(lambda row : make_mac_same(row), axis=1)

In [ ]:
def make_unique_id(row):
    return str(row['mac_address']) + '_' +  str(row['port_number'])

charging_2019['unique_plug_id'] = charging_2019.apply(lambda row : make_unique_id(row), axis=1)

In [ ]:
charging_2019['index1'] = charging_2019.index

In [ ]:
# change order of columns, put unique_plug_id in front, drop MAC ID
charging_2019 = charging_2019[['station_name',
                            'unique_plug_id', 
                            'start_date',
                            'end_date',
                            'total_duration',
                            'charging_time',
                            'parking_time',
                            'port_type',
                            'port_number',
                            'latitude',
                            'longitude',
                            'ended_by', 
                            'plug_in_event_id',
                            'driver_postal_code', 
                            'user_id',
                            'fee_bol', 
                            'weekday']]

In [ ]:
charging_2019 = charging_2019.sort_values(['unique_plug_id', 'start_date'])
charging_2019 = charging_2019.reset_index()
charging_2019.drop(['index'], inplace= True, axis= 1)
charging_2019['idle_time'] = None

In [ ]:
max_index = int(len(charging_2019) -1)

for index, row in charging_2019.iterrows():
    if index >= max_index:
        charging_2019['idle_time'][index] = None
    elif index <= max_index:
        if charging_2019['unique_plug_id'][index+1] != charging_2019['unique_plug_id'][index]:
            charging_2019['idle_time'][index] = None
        elif charging_2019['unique_plug_id'][index+1] == charging_2019['unique_plug_id'][index]:
            charging_2019['idle_time'][index] = charging_2019['start_date'][index+1] - charging_2019['end_date'][index]

In [ ]:
duration_cols = ["idle_time"]

for col in duration_cols:
     charging_2019[duration_cols] = charging_2019[duration_cols].apply(pd.to_timedelta, errors="coerce")

In [ ]:
# bring idle_time to front
charging_2019 = charging_2019[['station_name',
                            'unique_plug_id', 
                            'start_date',
                            'end_date',
                            'total_duration',
                            'charging_time',
                            'parking_time',
                            'idle_time',
                            'port_type',
                            'port_number',
                            'latitude',
                            'longitude',
                            'ended_by', 
                            'plug_in_event_id',
                            'driver_postal_code', 
                            'user_id',
                            'fee_bol', 
                            'weekday']]

### Potential Research Questions

Goal: Incentivize consumers in Germany (MUC and surroundings) to buy an EV by providing an optimal charging infrastructure (this means: there is (almost) always a charging station available at the right place (residential, leisure, work)

Identifying charging behavior patterns in relation to location specifics - Is Palo Alto a best practice example? If yes, what can other cities (e.g. MUC) learn from them in terms of charging infrastructure?

* Impact of COVID -- if time permits
* crisis 2013/2014 -- if time permits
* identify usage patterns according to location 
    * What are peak hours? Difference between weekdays / weekends?, per month? per year? 
    * peak times: How many PA users and how many from outside?
    * avg. charging time, avg. parking time, idle times
    * How many users are not from PA? What does MUC have to take into account in terms of "outside"/"shuttle" users?
    * Analyze recurring charging customers with zip codes not from PA? Which stations do they use? 
* What infrastructure strategy is PA using (e.g. how many new stations per year, where? (location characteristics); Is Palo Alto best practice? (define KPIs, what makes a best practice? Ideas: high idle times (even during peak hours or at least other free station nearby), high ratio of charging stations per EV, high coverage?)
* How does MUC compare?
* Which stations were added newly each year? compare with MUC

Potential categories: living, working, shopping, recreation

In [ ]:
def make_h3_code(row):
    return h3.geo_to_h3(
            lat=row['latitude'],
            lng=row['longitude'],
            resolution=8    # Average Hexagon Area (km2): 0.7373276
                            # Average Hexagon Edge Length (km)
                            # https://h3geo.org/docs/core-library/restable/
)
charging_2019['h3_code'] = charging_2019.apply(lambda row : make_h3_code(row), axis=1)
charging_2019.h3_code.nunique()